In [8]:
import pandas as pd # type: ignore
from tensorflow import keras # type: ignore
import numpy as np  # type: ignore 令人迷惑的在vscode使用jupyternotebook特有的不影响功能的问题
import tensorflow as tf
from small_functions import print_2d_data 

In [9]:
orig_data = pd.read_csv("data_set/从11个特征预测学生的测试分数.csv")
X , Y = orig_data.iloc[:,0:10] , orig_data.iloc[:,-1] #切片
X = X.drop(columns=["school","classroom","student_id"],axis=1)#按列丢弃无关项
X = pd.get_dummies(X,columns=["school_setting","school_type","teaching_method","gender","lunch"],dtype=int)#独热编码 #orig_data.iloc[0:5,] X.iloc[0:5,]

一层神经网络模型  
激活函数f(x)= x  
损失函数均方误差  
优化器Adam(默认比较优秀的优化器 结合了自适应梯度Adagrad和RMSProp防梯度急剧下降的优点)

In [21]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)#Sequential模型结构 多个网络层且无多余分支的堆叠 Dense就是常用的全连接层 运算就是output = activation(dot(input, kernel)+bias) units为该层的输出维度 而非输入维度  
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)) #adam（前期降低损失迅速，随迭代次数衰减学习率） 所以可以一开始把学习率提高一点 
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) #整个数据一共遍历epoch遍 每一次拿batch_size个样本来算损失 所以一个epoch需要算epoch/batch_size遍 更新同样次数的参数
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 649us/step - loss: 3065.4719
Epoch 2/100
54/54 [==============================] - 0s 574us/step - loss: 506.3102
Epoch 3/100
54/54 [==============================] - 0s 536us/step - loss: 74.8597
Epoch 4/100
54/54 [==============================] - 0s 515us/step - loss: 43.4522
Epoch 5/100
54/54 [==============================] - 0s 518us/step - loss: 40.1849
Epoch 6/100
54/54 [==============================] - 0s 530us/step - loss: 37.3628
Epoch 7/100
54/54 [==============================] - 0s 517us/step - loss: 34.5892
Epoch 8/100
54/54 [==============================] - 0s 510us/step - loss: 31.9350
Epoch 9/100
54/54 [==============================] - 0s 530us/step - loss: 29.4678
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 27.1894 - val_loss: 33.1946
Epoch 11/100
54/54 [==============================] - 0s 576us/step - loss: 25.1153
Epoch 12/100
54/54 [==============================] - 0s 571us/s

换个优化器 随机梯度下降

In [20]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.SGD(lr=0.01,clipvalue=0.5))#梯度裁剪 Gradient Clipping +-0.5
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 546us/step - loss: 466.9693
Epoch 2/100
54/54 [==============================] - 0s 509us/step - loss: 44.4441
Epoch 3/100
54/54 [==============================] - 0s 560us/step - loss: 28.8253
Epoch 4/100
54/54 [==============================] - 0s 474us/step - loss: 22.9786
Epoch 5/100
54/54 [==============================] - 0s 476us/step - loss: 19.7565
Epoch 6/100
54/54 [==============================] - 0s 413us/step - loss: 18.0407
Epoch 7/100
54/54 [==============================] - 0s 496us/step - loss: 16.7575
Epoch 8/100
54/54 [==============================] - 0s 463us/step - loss: 15.7532
Epoch 9/100
54/54 [==============================] - 0s 502us/step - loss: 14.9621
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 14.5011 - val_loss: 13.4467
Epoch 11/100
54/54 [==============================] - 0s 492us/step - loss: 13.9711
Epoch 12/100
54/54 [==============================] - 0s 459us/ste

自适应梯度 (对初始位置要求很严格 不然直接3000遍历还有1000损失 下降平缓均匀

In [18]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.Adagrad(lr=0.01,epsilon=1e-6))
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 539us/step - loss: 3602.5652
Epoch 2/100
54/54 [==============================] - 0s 572us/step - loss: 3520.6462
Epoch 3/100
54/54 [==============================] - 0s 573us/step - loss: 3467.9004
Epoch 4/100
54/54 [==============================] - 0s 541us/step - loss: 3425.6863
Epoch 5/100
54/54 [==============================] - 0s 558us/step - loss: 3389.3711
Epoch 6/100
54/54 [==============================] - 0s 486us/step - loss: 3357.3481
Epoch 7/100
54/54 [==============================] - 0s 505us/step - loss: 3328.2747
Epoch 8/100
54/54 [==============================] - 0s 491us/step - loss: 3301.5515
Epoch 9/100
54/54 [==============================] - 0s 547us/step - loss: 3276.7217
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 3253.3381 - val_loss: 2474.2014
Epoch 11/100
54/54 [==============================] - 0s 788us/step - loss: 3231.3008
Epoch 12/100
54/54 [=======================

结合梯度平方的指数移动平均数来调节学习率的变化。能够在不稳定（Non-Stationary）的目标函数情况下进行很好地收敛。递归神经网络时的一个良好选择？

In [22]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=1e-6)) #rho衰减系数 epsilon数值稳定性的小常数？
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 605us/step - loss: 10516.2354
Epoch 2/100
54/54 [==============================] - 0s 302us/step - loss: 9576.8496
Epoch 3/100
54/54 [==============================] - 0s 836us/step - loss: 8701.9951
Epoch 4/100
54/54 [==============================] - 0s 513us/step - loss: 7864.3018
Epoch 5/100
54/54 [==============================] - 0s 580us/step - loss: 7076.9102
Epoch 6/100
54/54 [==============================] - 0s 321us/step - loss: 6331.4673
Epoch 7/100
54/54 [==============================] - 0s 818us/step - loss: 5624.6172
Epoch 8/100
54/54 [==============================] - 0s 573us/step - loss: 4960.7485
Epoch 9/100
54/54 [==============================] - 0s 557us/step - loss: 4338.3154
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 3761.3542 - val_loss: 2748.6848
Epoch 11/100
54/54 [==============================] - 0s 766us/step - loss: 3224.8364
Epoch 12/100
54/54 [======================